In [1]:
from PIL import Image
import glob
import datetime
import re
from pathlib import Path
from shutil import copy2

In [2]:
images = glob.glob("Images/*")

In [3]:
def format1(s):
    for separator in ['/','-','_']:
        regex = r"\d{4}" +separator+ "\d{2}" +separator+ "\d{2}"
        match = re.search(regex, s)
        if match:
            try:
                date = datetime.datetime.strptime(match.group(), '%Y' +separator+ '%m' +separator+ '%d').date()
                return date
            except:
                print("Invalid date",regex)

In [4]:
def format2(s):
    regex = r'[_|-](\d{8})[_|-]'
    match = re.search(regex, s)
    if match:
        try:
            match = match.group()[1:-1]
            date = datetime.datetime.strptime(match, '%Y%m%d').date()
            return date
        except:
            print("Invalid date")

In [5]:
def format3(s):
    regex = r'_(\d{13}).'
    match = re.search(regex, s)
    if match:
        try:
            match = int(match.group()[1:-1])
            date = datetime.datetime.fromtimestamp(match/1000)
            return date
        except:
            print("Invalid date")

In [6]:
def get_date_taken(path):
    creation_date = None
    exif_data = None
    try:
        exif_data = Image.open(path)._getexif()
    except:
        pass
    if exif_data and '36867' in exif_data:
        creation_date = exif_data[36867]
        creation_date = datetime.datetime.strptime(creation_date, '%Y:%m:%d %H:%M:%S').date()
    else:
        creation_date = format1(path) or format2(path) or format3(path)
    return creation_date

In [7]:
err = []
succ = []
for i in range(len(images)):
    date = get_date_taken(images[i])
    if date:
        succ.append({ "path" : images[i],"date" : date})
    else:
        err.append(images[i])

In [8]:
def get_directory(d):
    return str(d.year)+'_'+str(d.month)

In [9]:
def create_directory(p):
    Path(p).mkdir(parents=True, exist_ok=True)

In [15]:
def copy_files(files):
    for image in files:
        directory = get_directory(image['date'])
        create_directory(directory)
        copy2(image['path'],directory)

In [16]:
%time copy_files(succ)

CPU times: user 2.01 s, sys: 5.15 s, total: 7.16 s
Wall time: 15.7 s
